In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sc

import sklearn
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold 

import statsmodels.api as sm
import matplotlib.pyplot as plt

C:\Users\soura\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
df = pd.read_csv('MART_data.csv')
df.shape

(4816, 12)

In [3]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
4811,FDZ39,19.700,Regular,0.018061,Meat,102.5990,OUT045,2002,NaN,Tier 2,Supermarket Type1,2063.9800
4812,FDY35,NaN,Regular,0.028062,Breads,44.0402,OUT019,1985,Small,Tier 1,Grocery Store,91.8804
4813,NCE06,5.825,Low Fat,0.091858,Household,160.7894,OUT018,2009,Medium,Tier 3,Supermarket Type2,1617.8940
4814,FDE57,NaN,Low Fat,0.036109,Fruits and Vegetables,140.6154,OUT027,1985,Medium,Tier 3,Supermarket Type3,4538.0930


In [4]:
df.dropna().shape # we cannot drop NA values as its looses 50% of its data 
#df.shape

(2632, 12)

In [5]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4816 entries, 0 to 4815
Data columns (total 12 columns):
Item_Identifier              4816 non-null object
Item_Weight                  4022 non-null float64
Item_Fat_Content             4816 non-null object
Item_Visibility              4816 non-null float64
Item_Type                    4816 non-null object
Item_MRP                     4816 non-null float64
Outlet_Identifier            4816 non-null object
Outlet_Establishment_Year    4816 non-null int64
Outlet_Size                  3426 non-null object
Outlet_Location_Type         4816 non-null object
Outlet_Type                  4816 non-null object
Item_Outlet_Sales            4816 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 451.6+ KB


In [7]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                   794
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1390
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [8]:
'''#calculating mean value for replacing null values in null value
avg_mean=df.pivot_table(values = 'Item_Weight',index ='Item_Identifier')
print("Average_Mean :",avg_mean)
def replace_null(cols):
    weight = cols[0]
    identifier =cols[1]
    if pd.isnull(weight):
        return  avg_mean['Item_Weight'][avg_mean.index==identifier]
    else:
        return weight
print("original number of null values",sum(df['Item_Weight'].isnull()))

df['Item_Weight'] = df[['Item_Weight','Item_Identifier']].apply(replace_null,axis=1)
print('final number of null values',df['Item_Identifier'].isnull().sum())'''
df['Item_Weight'] = df['Item_Weight'].fillna(method='ffill')
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
4811,FDZ39,19.700,Regular,0.018061,Meat,102.5990,OUT045,2002,NaN,Tier 2,Supermarket Type1,2063.9800
4812,FDY35,19.700,Regular,0.028062,Breads,44.0402,OUT019,1985,Small,Tier 1,Grocery Store,91.8804
4813,NCE06,5.825,Low Fat,0.091858,Household,160.7894,OUT018,2009,Medium,Tier 3,Supermarket Type2,1617.8940
4814,FDE57,5.825,Low Fat,0.036109,Fruits and Vegetables,140.6154,OUT027,1985,Medium,Tier 3,Supermarket Type3,4538.0930


In [9]:
#df['Item_Weight'] = df.Item_Weight.astype(float)
df.Item_Weight

0        9.300
1        5.920
2       17.500
3       19.200
4        8.930
         ...  
4811    19.700
4812    19.700
4813     5.825
4814     5.825
4815    12.850
Name: Item_Weight, Length: 4816, dtype: float64

### replacing null values of Outlet_size with mode because mode used when there is more frequent values

In [10]:
from scipy.stats import mode    # scipy = more mathematical functions than numpy
outlet_size_mode  = df.pivot_table(values='Outlet_Size',columns='Outlet_Type',aggfunc =lambda x:x.mode())
print(outlet_size_mode)
#replacing null values in outlet_size
def replace_outlet(cols):
    size=cols[0]
    Type = cols[1]
    if pd.isnull(size):
        return outlet_size_mode.loc['Outlet_Size'][outlet_size_mode.columns==Type][0]
    else:
        return size
print("original number of null values",sum(df['Outlet_Size'].isnull()))
df['Outlet_Size']=df[['Outlet_Size','Outlet_Type']].apply(replace_outlet,axis =1)
print('final data with null vakues',sum(df['Outlet_Size'].isnull()))


Outlet_Type Grocery Store Supermarket Type1 Supermarket Type2  \
Outlet_Size         Small             Small            Medium   

Outlet_Type Supermarket Type3  
Outlet_Size            Medium  
original number of null values 1390
final data with null vakues 0


In [11]:
df.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [12]:
print(df.Item_Type.describe())
print()
#df.head()
df['ItemType_Combined']= df['Item_Identifier'].apply(lambda x:x[0:2])
df['ItemType_Combined']= df['ItemType_Combined'].map({'FD':'Food','NC':'Non-Consumable','DR':'Drinks'})
print(df['ItemType_Combined'].value_counts())


count                      4816
unique                       16
top       Fruits and Vegetables
freq                        708
Name: Item_Type, dtype: object

Food              3484
Non-Consumable     866
Drinks             466
Name: ItemType_Combined, dtype: int64


In [13]:
#there was some typos in item_fat_content fixing it
df['Item_Fat_Content']=df['Item_Fat_Content'].replace({'LF':'Low fat','reg':'Regular','low fat':'Low fat'})
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,ItemType_Combined
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,Food
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,Drinks
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,Food
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,Food
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,Non-Consumable
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,FDZ39,19.700,Regular,0.018061,Meat,102.5990,OUT045,2002,Small,Tier 2,Supermarket Type1,2063.9800,Food
4812,FDY35,19.700,Regular,0.028062,Breads,44.0402,OUT019,1985,Small,Tier 1,Grocery Store,91.8804,Food
4813,NCE06,5.825,Low Fat,0.091858,Household,160.7894,OUT018,2009,Medium,Tier 3,Supermarket Type2,1617.8940,Non-Consumable
4814,FDE57,5.825,Low Fat,0.036109,Fruits and Vegetables,140.6154,OUT027,1985,Medium,Tier 3,Supermarket Type3,4538.0930,Food


### but again there are some non consumable product also so creating a new column for those



In [14]:
df.loc[df['ItemType_Combined']=="Non-Consumable",'Item_Fat_Content']='Non-Edible'
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,ItemType_Combined
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,Food
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,Drinks
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,Food
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,Food
4,NCD19,8.93,Non-Edible,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,Non-Consumable


In [15]:
df.Item_Visibility

0       0.016047
1       0.019278
2       0.016760
3       0.000000
4       0.000000
          ...   
4811    0.018061
4812    0.028062
4813    0.091858
4814    0.036109
4815    0.108633
Name: Item_Visibility, Length: 4816, dtype: float64

In [16]:
df[df['Item_Visibility']==0].head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,ItemType_Combined
3,FDX07,19.200,Regular,0.0,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,Food
4,NCD19,8.930,Non-Edible,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,Non-Consumable
5,FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,Food
10,FDY07,11.800,Low Fat,0.0,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0270,Food
32,FDP33,18.700,Low Fat,0.0,Snack Foods,256.6672,OUT018,2009,Medium,Tier 3,Supermarket Type2,3068.0060,Food


In [17]:
#This field mentions the percentage of total display area of all products in a store allocated to the particular product

Zero_Item_Visibility =(df['Item_Visibility']==0)
print ("All Zero items:",Zero_Item_Visibility.sum())

df.loc[Zero_Item_Visibility,'Item_Visibility']=df["Item_Visibility"].mean() 

Zero_Item_Visibility =(df['Item_Visibility']==0)
print ("Current Zero items :",Zero_Item_Visibility.sum())
print(df.Item_Visibility)

All Zero items: 294
Current Zero items : 0
0       0.016047
1       0.019278
2       0.016760
3       0.065810
4       0.065810
          ...   
4811    0.018061
4812    0.028062
4813    0.091858
4814    0.036109
4815    0.108633
Name: Item_Visibility, Length: 4816, dtype: float64


In [18]:
from sklearn.preprocessing  import LabelEncoder
Le = LabelEncoder()
#for outlet
df['outlet'] =Le.fit_transform(df['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','ItemType_Combined','Outlet_Type','outlet']
for i in var_mod:
    df[i]=Le.fit_transform(df[i])

### we have done something called as One-Hot-Coding -> creating a dummy variables one for each type of type variable



In [19]:
df.head(3)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,ItemType_Combined,outlet
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1,0,1,3735.1380,1,9
1,DRC01,5.92,3,0.019278,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.4228,0,3
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,1,0,1,2097.2700,1,9


In [20]:
df = pd.get_dummies(df,columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','ItemType_Combined','Outlet_Type','outlet'])
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Item_Fat_Content_3             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
ItemType_Combined_0            uint8
ItemType_Combined_1            uint8
ItemType_Combined_2            uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
outlet_0                       uint8
outlet_1                       uint8
o

In [21]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Item_Fat_Content_3             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
ItemType_Combined_0            uint8
ItemType_Combined_1            uint8
ItemType_Combined_2            uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
outlet_0                       uint8
outlet_1                       uint8
o

In [22]:
df.drop(['Item_Identifier','Outlet_Establishment_Year','Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)  #since it plays not much rol
df

,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Item_Outlet_Sales,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Outlet_Location_Type_0,...,outlet_0,outlet_1,outlet_2,outlet_3,outlet_4,outlet_5,outlet_6,outlet_7,outlet_8,outlet_9
0,9.300,0.016047,Dairy,249.8092,3735.1380,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,5.920,0.019278,Soft Drinks,48.2692,443.4228,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,17.500,0.016760,Meat,141.6180,2097.2700,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,19.200,0.065810,Fruits and Vegetables,182.0950,732.3800,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,8.930,0.065810,Household,53.8614,994.7052,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,19.700,0.018061,Meat,102.5990,2063.9800,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4812,19.700,0.028062,Breads,44.0402,91.8804,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
4813,5.825,0.091858,Household,160.7894,1617.8940,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4814,5.825,0.036109,Fruits and Vegetables,140.6154,4538.0930,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [23]:
df.drop(['Item_Type'],axis=1,inplace=True)  #since it plays not much rol
df

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Outlet_Location_Type_0,Outlet_Location_Type_1,...,outlet_0,outlet_1,outlet_2,outlet_3,outlet_4,outlet_5,outlet_6,outlet_7,outlet_8,outlet_9
0,9.300,0.016047,249.8092,3735.1380,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,5.920,0.019278,48.2692,443.4228,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,17.500,0.016760,141.6180,2097.2700,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,19.200,0.065810,182.0950,732.3800,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,8.930,0.065810,53.8614,994.7052,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,19.700,0.018061,102.5990,2063.9800,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0
4812,19.700,0.028062,44.0402,91.8804,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
4813,5.825,0.091858,160.7894,1617.8940,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4814,5.825,0.036109,140.6154,4538.0930,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [24]:
df.to_csv('clean_MART.csv')

In [25]:
Y=df['Item_Outlet_Sales']
df.drop(['Item_Outlet_Sales'],axis=1,inplace=True)
X=df

In [26]:
# Create train and test data
kf=sklearn.model_selection.KFold(n_splits=5,shuffle=True)
kf.get_n_splits(df)
df_train=[]
df_test=[]
for train_index, test_index in kf.split(df):
    df_train.append(train_index)
    df_test.append(test_index)


In [79]:
df

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,...,outlet_0,outlet_1,outlet_2,outlet_3,outlet_4,outlet_5,outlet_6,outlet_7,outlet_8,outlet_9
0,9.300,0.016047,249.8092,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,5.920,0.019278,48.2692,0,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,17.500,0.016760,141.6180,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,19.200,0.065810,182.0950,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,8.930,0.065810,53.8614,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,19.700,0.018061,102.5990,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4812,19.700,0.028062,44.0402,0,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4813,5.825,0.091858,160.7894,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4814,5.825,0.036109,140.6154,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [80]:
#print(df_test)
#print(train_index)
#print(test_index)

#df['Item_Weight'] = df['Item_Weight'].apply(pd.to_numeric)
#print(df.Item_Weight.isna().sum())
#print(df.dtypes)
#df

In [81]:
#df.to_csv('for_model_MART.csv')
df.dtypes

Item_Weight               float64
Item_Visibility           float64
Item_MRP                  float64
Item_Fat_Content_0          uint8
Item_Fat_Content_1          uint8
Item_Fat_Content_2          uint8
Item_Fat_Content_3          uint8
Outlet_Location_Type_0      uint8
Outlet_Location_Type_1      uint8
Outlet_Location_Type_2      uint8
Outlet_Size_0               uint8
Outlet_Size_1               uint8
Outlet_Size_2               uint8
ItemType_Combined_0         uint8
ItemType_Combined_1         uint8
ItemType_Combined_2         uint8
Outlet_Type_0               uint8
Outlet_Type_1               uint8
Outlet_Type_2               uint8
Outlet_Type_3               uint8
outlet_0                    uint8
outlet_1                    uint8
outlet_2                    uint8
outlet_3                    uint8
outlet_4                    uint8
outlet_5                    uint8
outlet_6                    uint8
outlet_7                    uint8
outlet_8                    uint8
outlet_9      

In [82]:
for i in range(5):
    x=[]
    y=[]
    x1=[]
    y1=[]
    for j in range(24):
        #print(j,df_train[i][j])
        x.append(X.iloc[df_train[i][j]])
        y.append(Y.iloc[df_train[i][j]])
    for k in range(6):
        x1.append(X.iloc[df_test[i][k]])
        y1.append(Y.iloc[df_test[i][k]])
    regr = linear_model.LinearRegression()
    regr.fit(x, y)
    result_train = regr.predict(x)
    result_test = regr.predict(x1)
#     plt.plot(y,result_train,'r+')
#     plt.show()
#     plt.plot(y1,result_test,'b+')
#     plt.show()
    MSE_train=mean_squared_error(y,result_train)
    MSE_test=mean_squared_error(y1, result_test)
    print("MSE of training set is ", MSE_train) 
    print("MSE of testing set is ", MSE_test) 
    print("")
    avg_train = []
    avg_test = []
    avg_train.append(MSE_train)
    avg_test.append(MSE_test)
print("AVERGAE TRAIN MSE :",np.average(avg_train))
print("AVERGAE TEST MSE :",np.average(avg_test))
print()
print("AVERGAE TRAIN RMSE :",np.sqrt(np.average(avg_train)))
print("AVERGAE TEST RMSE :",np.sqrt(np.average(avg_test)))
#print(x,y,x1,y1)

        

MSE of training set is  1743105.0261878
MSE of testing set is  3654612.572253758

MSE of training set is  1575273.494618622
MSE of testing set is  4519049.385434971

MSE of training set is  2117274.2612433606
MSE of testing set is  2254367.561746297

MSE of training set is  2278558.6031480916
MSE of testing set is  3092121.853238364

MSE of training set is  1339237.9051266278
MSE of testing set is  6089877.591725866

AVERGAE TRAIN MSE : 1339237.9051266278
AVERGAE TEST MSE : 6089877.591725866

AVERGAE TRAIN RMSE : 1157.2544686138083
AVERGAE TEST RMSE : 2467.7677345580696


In [36]:
dfx = pd.read_csv('clean_MART.csv')
dfx

,Unnamed: 0,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Outlet_Location_Type_0,...,outlet_0,outlet_1,outlet_2,outlet_3,outlet_4,outlet_5,outlet_6,outlet_7,outlet_8,outlet_9
0,0,9.300,0.016047,249.8092,3735.1380,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1,5.920,0.019278,48.2692,443.4228,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,2,17.500,0.016760,141.6180,2097.2700,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,3,19.200,0.065810,182.0950,732.3800,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,4,8.930,0.065810,53.8614,994.7052,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,4811,19.700,0.018061,102.5990,2063.9800,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4812,4812,19.700,0.028062,44.0402,91.8804,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
4813,4813,5.825,0.091858,160.7894,1617.8940,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4814,4814,5.825,0.036109,140.6154,4538.0930,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [50]:
seed = 10
test_size = 0.3
trainingSet, testingSet = train_test_split(dfx, test_size = test_size, random_state = seed)

In [51]:
trainingSet.shape , testingSet.shape

((3371, 32), (1445, 32))

In [52]:
Y =trainingSet['Item_Outlet_Sales']
trainingSet = trainingSet.drop(['Item_Outlet_Sales'],axis=1)
X = trainingSet
X.shape,Y.shape

((3371, 31), (3371,))

In [56]:
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     259.4
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        16:44:39   Log-Likelihood:                -28441.
No. Observations:                3371   AIC:                         5.692e+04
Df Residuals:                    3353   BIC:                         5.703e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -44

In [57]:
lm_result.mse_model

325024684.11367077

In [58]:
dfx.columns

Index(['Unnamed: 0', 'Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Item_Outlet_Sales', 'Item_Fat_Content_0', 'Item_Fat_Content_1',
       'Item_Fat_Content_2', 'Item_Fat_Content_3', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2', 'ItemType_Combined_0',
       'ItemType_Combined_1', 'ItemType_Combined_2', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9'],
      dtype='object')

In [65]:
X

,const,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,...,outlet_0,outlet_1,outlet_2,outlet_3,outlet_4,outlet_5,outlet_6,outlet_7,outlet_8,outlet_9
0,1.0,9.300,0.016047,249.8092,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1.0,5.920,0.019278,48.2692,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,1.0,17.500,0.016760,141.6180,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1.0,19.200,0.065810,182.0950,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,1.0,8.930,0.065810,53.8614,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4811,1.0,19.700,0.018061,102.5990,0,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4812,1.0,19.700,0.028062,44.0402,0,0,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4813,1.0,5.825,0.091858,160.7894,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4814,1.0,5.825,0.036109,140.6154,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [66]:
X = trainingSet[['Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Fat_Content_1',
       'Item_Fat_Content_2', 'Item_Fat_Content_3', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2', 'ItemType_Combined_0',
       'ItemType_Combined_1', 'ItemType_Combined_2', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     275.4
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        16:57:37   Log-Likelihood:                -28442.
No. Observations:                3371   AIC:                         5.692e+04
Df Residuals:                    3354   BIC:                         5.702e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -64

In [67]:
X = trainingSet[[ 'Item_Visibility', 'Item_MRP', 'Item_Fat_Content_1',
       'Item_Fat_Content_2', 'Item_Fat_Content_3', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2', 'ItemType_Combined_0',
       'ItemType_Combined_1', 'ItemType_Combined_2', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     293.8
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        16:58:15   Log-Likelihood:                -28442.
No. Observations:                3371   AIC:                         5.692e+04
Df Residuals:                    3355   BIC:                         5.701e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -57

In [68]:
X = trainingSet[[ 'Item_Visibility', 'Item_MRP',
       'Item_Fat_Content_2', 'Item_Fat_Content_3', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2', 'ItemType_Combined_0',
       'ItemType_Combined_1', 'ItemType_Combined_2', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     314.9
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        16:58:47   Log-Likelihood:                -28442.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3356   BIC:                         5.701e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -58

In [69]:
X = trainingSet[[ 'Item_Visibility', 'Item_MRP',
        'Item_Fat_Content_3', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2', 'ItemType_Combined_0',
       'ItemType_Combined_1', 'ItemType_Combined_2', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     314.9
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        16:59:30   Log-Likelihood:                -28442.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3356   BIC:                         5.701e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -60

In [70]:
X = trainingSet[[ 'Item_Visibility', 'Item_MRP',
        'Item_Fat_Content_3', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2', 'ItemType_Combined_0',
       'ItemType_Combined_1', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     314.9
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:00:04   Log-Likelihood:                -28442.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3356   BIC:                         5.701e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -74

In [71]:
X = trainingSet[[ 'Item_Visibility', 'Item_MRP',
        'Item_Fat_Content_3', 'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2',
       'ItemType_Combined_1', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     339.2
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:00:29   Log-Likelihood:                -28442.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3357   BIC:                         5.700e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -79

In [72]:
X = trainingSet[[ 'Item_Visibility', 'Item_MRP',
         'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2',
       'ItemType_Combined_1', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     367.4
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:01:06   Log-Likelihood:                -28442.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3358   BIC:                         5.699e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -79

In [73]:
X = trainingSet[[  'Item_MRP',
         'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2',
       'ItemType_Combined_1', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5', 'outlet_6',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     400.6
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:01:32   Log-Likelihood:                -28443.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3359   BIC:                         5.698e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -94

In [74]:
X = trainingSet[[  'Item_MRP',
         'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2',
       'ItemType_Combined_1', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_2', 'outlet_3', 'outlet_4', 'outlet_5',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     400.6
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:02:07   Log-Likelihood:                -28443.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3359   BIC:                         5.698e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -84

In [75]:
X = trainingSet[[  'Item_MRP',
         'Outlet_Location_Type_0',
       'Outlet_Location_Type_1', 'Outlet_Location_Type_2', 'Outlet_Size_0',
       'Outlet_Size_1', 'Outlet_Size_2',
       'ItemType_Combined_1', 'Outlet_Type_0',
       'Outlet_Type_1', 'Outlet_Type_2', 'Outlet_Type_3', 'outlet_0',
       'outlet_1', 'outlet_3', 'outlet_4', 'outlet_5',
       'outlet_7', 'outlet_8', 'outlet_9']]
X = sm.add_constant(X)
lm = smf.OLS(Y,X)
lm_result = lm.fit()
print(lm_result.summary())

                            OLS Regression Results                            
Dep. Variable:      Item_Outlet_Sales   R-squared:                       0.567
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     440.5
Date:                Sun, 09 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:02:35   Log-Likelihood:                -28443.
No. Observations:                3371   AIC:                         5.691e+04
Df Residuals:                    3360   BIC:                         5.698e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -78

In [78]:
np.sqrt(lm_result.mse_model)

23490.691805202026